We use the data used in the analysis reported on in (Jego et al, 2022)

In [20]:
import APEX as ap
import pyccl as ccl
import numpy as np
import sacc as sc

cosmo = ccl.Cosmology(
    Omega_c=0.261,
    Omega_b=0.049,
    h=0.677,
    n_s=0.9665,
    sigma8=0.8102,
    #transfer_function="bbks",
    #matter_power_spectrum="halofit")
)

import warnings

# The warning message to ignore
warning_message = "The FITS format without the 'sacc_ordering' column is deprecated."

# Apply the filter to ignore this specific UserWarning
warnings.filterwarnings("ignore", message=warning_message, category=UserWarning)

# ... the rest of your code that uses the sacc library

One should check the layout of the sacc file first:

In [21]:
sacc_data = sc.Sacc.load_fits('cls_cov_minimal_newcov.fits')

#print all tracers
print("Tracer combinations:", sacc_data.get_tracer_combinations())

Tracer combinations: [(np.str_('CIBLenz__0'), np.str_('eBOSSQSO__0')), (np.str_('CIBLenz__0'), np.str_('eBOSSQSO__1')), (np.str_('CIBLenz__1'), np.str_('eBOSSQSO__0')), (np.str_('CIBLenz__1'), np.str_('eBOSSQSO__1')), (np.str_('CIBLenz__2'), np.str_('eBOSSQSO__0')), (np.str_('CIBLenz__2'), np.str_('eBOSSQSO__1')), (np.str_('DELS__0'), np.str_('CIBLenz__0')), (np.str_('DELS__0'), np.str_('CIBLenz__1')), (np.str_('DELS__0'), np.str_('CIBLenz__2')), (np.str_('DELS__0'), np.str_('DELS__0')), (np.str_('DELS__1'), np.str_('CIBLenz__0')), (np.str_('DELS__1'), np.str_('CIBLenz__1')), (np.str_('DELS__1'), np.str_('CIBLenz__2')), (np.str_('DELS__1'), np.str_('DELS__1')), (np.str_('DELS__2'), np.str_('CIBLenz__0')), (np.str_('DELS__2'), np.str_('CIBLenz__1')), (np.str_('DELS__2'), np.str_('CIBLenz__2')), (np.str_('DELS__2'), np.str_('DELS__2')), (np.str_('DELS__3'), np.str_('CIBLenz__0')), (np.str_('DELS__3'), np.str_('CIBLenz__1')), (np.str_('DELS__3'), np.str_('CIBLenz__2')), (np.str_('DELS__3'

In [22]:
gd_file = 'cls_cov_minimal_newcov.fits'

cib_snu_file = 'filtered_snu_planck.fits'

In [23]:
gdwsp = ap.GalaxyDensityTracerWorkspace(
    sacc_file=gd_file,
    tracer_name_root="DELS__",
    max_index=3,
    cosmology=cosmo
)

cibwsp = ap.CIBIntensityTracerWorkspace(
    flux_fits_file=cib_snu_file,
    cosmology=cosmo,
    tracer_name_root="CIBLenz__",
    single_index=2
)

tracer_combos = [('DELS__0', 'DELS__0'),
                 ('DELS__1', 'DELS__1'),
                 ('DELS__2', 'DELS__2'),
                 ('DELS__3', 'DELS__3'),
                 ('DELS__0', 'CIBLenz__2'),
                 ('DELS__1', 'CIBLenz__2'),
                 ('DELS__2', 'CIBLenz__2'),
                 ('DELS__3', 'CIBLenz__2'),
                ]

s = ap.SaccWorkspace(gd_file, tracer_combinations=tracer_combos)


In [24]:
mmodel = ap.MaleubreModel(
    Tracer1Workspace=gdwsp,
    Tracer2Workspace=cibwsp,
    tracer_combos=tracer_combos,
    sacc_workspace=s,
    cosmology=cosmo,
    logged_N=True,
    k_max = 0.3
)

In [ ]:
mcmc = ap.MCMCWorkspace(
    model=mmodel,
    likelihood_function='lightweight_log_likelihood_function',
    sacc_file=gd_file

)

params = ['b_g0', 'b_g1', 'b_g2', 'b_g3',
          'N_gg0', 'N_gg1', 'N_gg2', 'N_gg3',
          'A_gg0', 'A_gg1', 'A_gg2', 'A_gg3',
          'N_gnu0', 'N_gnu1', 'N_gnu2', 'N_gnu3',
          'A_gnu0', 'A_gnu1', 'A_gnu2', 'A_gnu3',
          'bpsfr0', 'bpsfr1', 'bpsfr2', 'bpsfr3']
mcmc.set_param_priors(
    params=params,
    priors=[
        (0, 4), (0, 4), (0, 4), (0, 4),
        (-12, -4), (-12, -4), (-12, -4), (-12, -4),
        (-100, 100), (-100, 100), (-100, 100), (-100, 100),
        (-15, -8), (-15, -8), (-15, -8), (-15, -8),
        (-100, 100), (-100, 100), (-100, 100), (-100, 100),
        (0, 1), (0, 1), (0, 1), (0, 1),
    ]
)

mcmc.set_param_references(
    params=params,
    references=[
        0.8, 0.9, 1.2, 1.5,
        -8, -8, -8, -8,
        7, 7, 7, 7,
        -12, -12, -12, -12,
        7, 7, 7, 7,
        0.059, 0.11, 0.15, 0.18,
    ]
)

mcmc.set_param_proposals(
    params=params,
    proposals=[
        0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1,
        1, 1, 1, 1,
        0.1, 0.1, 0.1, 0.1,
        1, 1, 1, 1,
        0.003, 0.003, 0.003, 0.003,
    ]
)

mcmc.set_grouped_params({
    'b_gs': ['b_g0', 'b_g1', 'b_g2', 'b_g3'],
    'N_ggs': ['N_gg0', 'N_gg1', 'N_gg2', 'N_gg3'],
    'A_ggs': ['A_gg0', 'A_gg1', 'A_gg2', 'A_gg3'],
    'N_gnus': ['N_gnu0', 'N_gnu1', 'N_gnu2', 'N_gnu3'],
    'A_gnus': ['A_gnu0', 'A_gnu1', 'A_gnu2', 'A_gnu3'],    
    'bpsfrs': ['bpsfr0', 'bpsfr1', 'bpsfr2', 'bpsfr3']
})

mcmc.MCMC_config(params, sampler_info={'minimize': {'seed':42}})

sampler = mcmc.minimize_run()


[<bound method maleubremodel.log_likelihood_function of <apex.maleubremodel object at 0x7f1bdf033ed0>>] Initialized external likelihood.
[minimize] *WARNING* This run has been SEEDED with seed 42
[minimize] Initializing
[minimize] Run 1/2 will start from random initial point:
[minimize] {'b_g0': np.float64(0.8), 'b_g1': np.float64(0.9), 'b_g2': np.float64(1.2), 'b_g3': np.float64(1.5), 'N_gg0': np.float64(-8.0), 'N_gg1': np.float64(-8.0), 'N_gg2': np.float64(-8.0), 'N_gg3': np.float64(-8.0), 'A_gg0': np.float64(7.0), 'A_gg1': np.float64(7.0), 'A_gg2': np.float64(7.0), 'A_gg3': np.float64(7.0), 'N_gnu0': np.float64(-12.0), 'N_gnu1': np.float64(-12.0), 'N_gnu2': np.float64(-12.0), 'N_gnu3': np.float64(-12.0), 'A_gnu0': np.float64(7.0), 'A_gnu1': np.float64(7.0), 'A_gnu2': np.float64(7.0), 'A_gnu3': np.float64(7.0), 'bpsfr0': np.float64(0.059), 'bpsfr1': np.float64(0.11), 'bpsfr2': np.float64(0.15), 'bpsfr3': np.float64(0.18)}
[minimize] Run 2/2 will start from random initial point:
[mini